# Export d’articles MDB

Ce notebook parcourt un export HTML de MDB et le transforme en un tableau d’articles. On peut transformer ce tableau en CSV ou json par exemple.

Pour mieux faire: l’API de DCX https://github.com/digicol/dcx-sdk-php

In [1]:
from bs4 import BeautifulSoup
from glob import glob
import pandas as pd
import re
import gender_guesser.detector as gender
from nameparser import HumanName
import unicodedata

In [2]:
pd.set_option('display.max_colwidth', 999)

In [3]:
files = glob('stories/*.html')

In [4]:
def getContents(file):
    with open(file, 'r') as f:
        doc = BeautifulSoup(f.read(), 'html.parser')
    meta = {}
    for tr in doc.select('table tr'):
        meta[tr.find('th').text.replace(':', '').strip()] = tr.find('td').text.strip()
    meta['Paragraphs'] = [i.text for i in doc.select('p') if i != '']
    return meta

In [5]:
cols = ['Titre', 'Chapeau', 'Pour Correction (LA)', 'Paragraphs']
df = pd.DataFrame(columns=cols)
for file in files:
    data = getContents(file)
    story = {}
    for key in cols:
        if key in data.keys():
            story[key] = data[key]
        else:
            print('WARNING: no', key, 'for story', data['Titre'])
            story[key] = ''
    df = df.append(pd.DataFrame([story], columns=cols))
print('Dataframe contains', len(df), 'stories.')

Dataframe contains 102 stories.


In [6]:
df.reset_index(inplace=True)

In [7]:
def getText(paragraph):
    p = [i.strip() for i in paragraph[1:-1]]
    return "\n".join(p).strip()

In [8]:
df['Text'] = df['Paragraphs'].apply(getText)